In [0]:
!pip install hana_ml mlflow
dbutils.library.restartPython()

In [0]:
import hana_ml
from hana_ml import dataframe
import mlflow
print(hana_ml.__version__)
print(mlflow.__version__    )

In [0]:
from hana_ml.algorithms.pal.utility import DataSets, Settings
import os
scope = "<scope>" #as defined in create_secrets
os.environ["hana_url"] = dbutils.secrets.get(scope=scope, key="hana_url")
os.environ["hana_port"] = dbutils.secrets.get(scope=scope, key="hana_port")
os.environ["hana_user"] = dbutils.secrets.get(scope=scope, key="hana_user")
os.environ["hana_password"] = dbutils.secrets.get(scope=scope, key="hana_password")

connection_context = dataframe.ConnectionContext(os.environ["hana_url"] ,os.environ["hana_port"] ,os.environ["hana_user"] , os.environ["hana_password"])
connection_context.connection.isconnected()


### Define the model input and output signature
Here the interface is the user provides 
- the name of the HANA Cloud inference table (this could be the same or another HANA Cloud instance where the training was done)
- the output is the prediction for each row in the inference table identified by "ID"

In [0]:
from mlflow.models import ModelSignature, infer_signature
from mlflow.types.schema import Schema, ColSpec
signature = ModelSignature(inputs = Schema([ColSpec("string", "INFERENCE_TABLE_NAME")]))

signature.outputs = Schema([ColSpec("integer", "ID"), ColSpec("double", "SCORES")])

#### Create the custom pyfunc model for hana-ml

In [0]:
# Save as script: hana_ml_pyfunc_model.py
# %%writefile "./hana_ml_pyfunc_model.py"
import mlflow
from mlflow import pyfunc
from mlflow.models import set_model
import hana_ml
from hana_ml import dataframe
from hana_ml.model_storage import ModelStorage
import os

class CustomException(Exception):
    """Exception raised to get messages
    Attributes:
        message -- explanation of the error
    """
    def __init__(self, message="HANA ML Pyfunc Custom Exception"):
        self.message = message
        super().__init__(self.message)

class hana_ml_pyfunc_model(pyfunc.PythonModel):

    def connectToHANA(self, context):
        try:
            url =  os.getenv('hana_url') 
            port = os.getenv('hana_port')
            user = os.getenv('hana_user')
            passwd = os.getenv('hana_password')
            connection_context = dataframe.ConnectionContext(url, port, user, passwd)
            return connection_context
        except Exception as e:
            print(f"Exception occurred: {e}")
            raise e
            return "Exception:{e}", e
    @mlflow.trace
    def load_context(self, context):
        try: 
            with mlflow.start_span("load_context"):
                self.model = context.artifacts["model"]
                self.connection_context = self.connectToHANA(context)
                print("HANA_ML_MODEL loaded in load_context")
        except Exception as e:
            print(f"Exception occurred: {e}")
            raise Exception(f"Loading the context failed due to {e}")
    @mlflow.trace
    def predict(self, context, model_input):
        table_name = None
        try: 
            if self.connection_context.connection.isconnected() == False:
                with mlflow.start_span("connect_to_HANA"):
                    self.connection_context = self.connectToHANA(context)
                    if self.connection_context.connection.isconnected():
                        print("HANA Connection Successful")
                    else:
                        raise Exception("HANA Connection Failed")
       
            with mlflow.start_span("load_model"):
                hana_model = ModelStorage.load_mlflow_model(connection_context=self.connection_context, model_uri=self.model,use_temporary_table=False, force=True)
           
                print("HANA_ML_MODEL loaded in predict")
                print("model_input", model_input)
                table_name = str(model_input["INFERENCE_TABLE_NAME"][0]) 
                print("Table Name:", table_name)
            with mlflow.start_span("hana_ml_predict"):
                df = self.connection_context.table(table_name)
                if df.count() > 0:
                  
                    print(f"Running HANA ML inference on {table_name} with {df.count()} records")
                    prediction = hana_model.predict(df, key = "ID").collect()
                    print("Prediction completed")
                    return  prediction
                else:
                    raise Exception(f"HANA Inference Table {table_name} is empty")
        
            
        except Exception as e:
        
            print(f"Exception occurred: {e}")
            raise f"Exception:{e}"
        
set_model(hana_ml_pyfunc_model())

### Log the custom pyfunc model

In [0]:
import mlflow
mlflow.set_tracking_uri("databricks")
runid='<selected runid which has the chosen model>' #this is the run_id from training run in 1_hana_mlflow_ar_bike_test_train
model_uri='runs:/{}/model'.format(runid)

experiment_name = '<experiment_name>' 

mlflow.set_experiment(experiment_name)
model_file = "hana_ml_pyfunc_model.py"

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=model_file,
            artifacts={"model": model_uri},
            pip_requirements=["hana-ml","ipython"],
            signature = signature,
            input_example={"INFERENCE_TABLE_NAME" : "INFERENCE_BIKE_DATA_TBL"},
           
    )
# Register the model
model_uri = f"runs:/{run.info.run_id}/model"
registered_model_name = "<registered_model_name>"
mlflow.register_model(model_uri=model_uri, name=registered_model_name)

#### Test the model

In [0]:
run_id = "<run_id of model logged above>" #run.info.run_id
logged_model = f'runs:/{run_id}/model'
dataset = {"inputs": {"INFERENCE_TABLE_NAME" : "INFERENCE_BIKE_DATA_TBL"}}
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model.predict(dataset["inputs"])

#### Test the model endpoint

In [0]:
!pip install uv

In [0]:

run_id = "<run_id of model logged above>" #run.info.run_id
model_uri = f"runs:/{run_id}/model"
dataset = {"inputs": {"INFERENCE_TABLE_NAME" : "INFERENCE_BIKE_DATA_TBL"}}
input_data = dataset

mlflow.models.predict(
    model_uri=model_uri,
    input_data=dataset["inputs"],
    env_manager="uv",
)

#### Now we can deploy the registered model following steps here [Model Serving](https://docs.databricks.com/aws/en/machine-learning/model-serving/store-env-variable-model-serving?language=Serving%C2%A0UI)

In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")
endpoint = client.create_endpoint(
    name="hana-mlflow-serving",
    config={
        "served_entities": [
            {
                "name": "hana-ml-ar",
                "entity_name": registered_model_name,
                "entity_version": "1",
                "workload_size": "Small",
                "scale_to_zero_enabled": True,
                "environment_vars": {
                    "ENABLE_MLFLOW_TRACING": "true",
                    "hana_url": os.environ["hana_url"],
                    "hana_port": os.environ["hana_port"],
                    "hana_user": os.environ["hana_user"],
                    "hana_password": os.environ["hana_password"]
                }
            }
        ]
    }
)